# GPU Preparation

GPU and nvcc (aka cuda) versions should be within the same major version. I've noticed that Ubuntu 22.04 loads on some systems have been way out of **alignment**. Try to get them at the same version.


```
sudo apt-get purge 'nvidia*' 'cuda*'

sudo apt-get install nvidia-driver-535

sudo reboot

wget https://developer.download.nvidia.com/compute/cuda/12.2.0/local_installers/cuda_12.2.0_535.54.03_linux.run

chmod a+x cuda_12.2.0_535.54.03_linux.run
 
sudo ./cuda_12.2.0_535.54.03_linux.run # And follow the prompts

# Edit your .bashrc and put these in. But don't put the hastags in front of them.
# export PATH=/usr/local/cuda-12.2/bin${PATH:+:${PATH}}
#export LD_LIBRARY_PATH=/usr/local/cuda-12.2/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}


# I also had to do this.  If you cannot type nvcc --version then you need to check the permissions.
sudo chmod -R 755 /usr/local/cuda-12.2


```

The results should be something like this:

```
acshell@ip-10-114-92-249:~$ nvidia-smi | grep -i "cuda version" | awk '{print $9}'
12.2
acshell@ip-10-114-92-249:~$ nvcc --version
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Jun_13_19:16:58_PDT_2023
Cuda compilation tools, release 12.2, V12.2.91
Build cuda_12.2.r12.2/compiler.32965470_0

```

In [7]:

import torch
print(f"Is Cuda available? {torch.cuda.is_available()}.")  # Should return True
print(f"Torch Version is {torch.version.cuda}.")  # Should return '12.1'

import time

Is Cuda available? True.
Torch Version is 12.1.


### Torch Version

PyTorch should be within a minor version of the cuda drivers and cuda drivers need to align with nvidia drivers.  Try hard to make this happen.

Here's an example.

In [6]:
import torch
import time

# Define the size of the tensors
size = 10000

# Create two large random tensors for CPU
tensor1_cpu = torch.randn(size, size)
tensor2_cpu = torch.randn(size, size)

# Perform matrix multiplication on the CPU and time it
start_time = time.time()
result_cpu = torch.matmul(tensor1_cpu, tensor2_cpu)
end_time = time.time()

print(f"Matrix multiplication on CPU took: {end_time - start_time:.4f} seconds")
print(f"Result tensor size on CPU: {result_cpu.size()}")

# Check if CUDA is available and perform the same test on the GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    
    # Create two large random tensors for GPU
    tensor1_gpu = tensor1_cpu.to(device)
    tensor2_gpu = tensor2_cpu.to(device)

    # Perform matrix multiplication on the GPU and time it
    torch.cuda.synchronize()  # Ensure all CUDA operations are finished
    start_time = time.time()
    result_gpu = torch.matmul(tensor1_gpu, tensor2_gpu)
    torch.cuda.synchronize()  # Ensure the GPU has finished the computation
    end_time = time.time()

    print(f"Matrix multiplication on GPU took: {end_time - start_time:.4f} seconds")
    print(f"Result tensor size on GPU: {result_gpu.size()}")
else:
    print("CUDA is not available on this system.")


Matrix multiplication on CPU took: 6.3072 seconds
Result tensor size on CPU: torch.Size([10000, 10000])
Matrix multiplication on GPU took: 0.1868 seconds
Result tensor size on GPU: torch.Size([10000, 10000])
